# Requests and Responses

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK

This notebook demonstrates how agents interact with the PrimAITE simulation via the Request system.


In [1]:
!primaite setup

2025-03-24 09:52:23,803: Performing the PrimAITE first-time setup...
2025-03-24 09:52:23,803: Building the PrimAITE app directories...
2025-03-24 09:52:23,803: Building primaite_config.yaml...
2025-03-24 09:52:23,803: Rebuilding the demo notebooks...
2025-03-24 09:52:23,826: Rebuilding the example notebooks...
2025-03-24 09:52:23,827: PrimAITE setup complete!


In [2]:
from primaite.simulator.network.hardware.node_operating_state import NodeOperatingState
from primaite.simulator.network.hardware.nodes.host.host_node import HostNode
from primaite.simulator.sim_container import Simulation


## Sending a request

Before we can send some requests we need to set up a minimal network simulation. The code snippet below creates a PrimAITE simulation with a singular generic host called `client`.

In [3]:
sim = Simulation()

sim.network.add_node(
    HostNode.from_config(
        config = {
                'type': "host-node",
                'hostname': "client",
                'ip_address': '10.0.0.1',
                'subnet_mask': '255.255.255.0',
                'operating_state': "ON",
                }
    )
)
client = sim.network.get_node_by_hostname('client')


Now we can simulation component to interact with, we can start sending some requests.

A request is structured in a similar way to a command line interface - a list of strings with positional args. It's also possible to supply an optional `context` dictionary. We will craft a request that stops the pre-installed `dns-client` service on the client node.

First let's verify that the `dns-client` is running on the client.

In [4]:
client.software_manager.show()
client.software_manager.software['dns-client'].operating_state.name

+---------------------------------------------------------------------------------------+
|                                client Software Manager                                |
+----------------------+-------------+-----------------+--------------+------+----------+
| Name                 | Type        | Operating State | Health State | Port | Protocol |
+----------------------+-------------+-----------------+--------------+------+----------+
| arp                  | Service     | RUNNING         | GOOD         | 219  | udp      |
| icmp                 | Service     | RUNNING         | GOOD         | None | icmp     |
| dns-client           | Service     | RUNNING         | GOOD         | 53   | tcp      |
| ntp-client           | Service     | RUNNING         | GOOD         | 123  | udp      |
| web-browser          | Application | CLOSED          | GOOD         | 80   | tcp      |
| nmap                 | Application | CLOSED          | GOOD         | None | none     |
| user-ses

'RUNNING'

Send a request to the simulator to stop the `dns-client`.

In [5]:
response = sim.apply_request(
    request=["network", "node", "client", "service", "dns-client", "stop"],
    context={}
    )
print(response)

status='success' data={}



The request returns a `RequestResponse` object which tells us that the request was successfully executed. Let's verify that the `dns-client` is in a stopped state now.

In [6]:
print(f"DNS Client state: {client.software_manager.software.get('dns-client').operating_state.name}")

DNS Client state: STOPPED


## Unreachable requests

If we attempt to send a request to something that doesn't exist, we will get an unreachable request status.

In [7]:
response = sim.apply_request(
    request=["network", "node", "client", "service", "non-existent-application", "stop"],
    context={}
    )
print(response)

status='unreachable' data={'reason': ("Request ['non-existent-application', 'stop'] could not be processed because non-existent-application is not a valid request name", 'within this RequestManager')}


## Failed requests

Sometimes requests cannot be executed by the simulation. For example if we turn off the client node, we cannot execute the software that is running on it.

In [8]:
response = sim.apply_request(
    request = ["network", "node", "client", "shutdown"],
    context = {}
)
print(response)

status='success' data={}


We need to apply timestep a few times for the client to go from `SHUTTING_DOWN` to `OFF` state.

In [9]:
print(f"client is in state: {client.operating_state.name}")
sim.apply_timestep(1)
sim.apply_timestep(2)
sim.apply_timestep(3)
sim.apply_timestep(4)
print(f"client is in state: {client.operating_state.name}")

client is in state: SHUTTING_DOWN
client is in state: OFF


Now, if we try to start the `dns-client` back up, we get a failure because we cannot start software on a node that is turned off.

In [10]:
response = sim.apply_request(
    request=["network", "node", "client", "service", "dns-client", "start"],
    context={}
    )
print(response)

status='failure' data={'reason': "Cannot perform request on node 'client' because it is not powered on."}
